In [5]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, display_markdown
from IPython.display import Javascript

In [1]:
eps = 1.0e-10
R = 8.3145e-3

def generate_positions(N):

    # Special cases: 
    
    if N==0:
        return np.array([], dtype=complex)
    
    if N==1:
        return np.array([1], dtype=complex)
    
    if N==2:
        return np.array([[1,2]], dtype=complex)
    
    # For N>2, we proceed iteratively:
    
    # Initially we have two beads on the x-axis
    PosArray = np.array([[1,2]], dtype=complex)

    # Now add beads one at a time
    for n in range(2, N):

        # Last column stores the position of the last bead added
        LastPos = PosArray[:,-1]

        # Second-last column stores the position of the second-to-last bead
        PenUltPos = PosArray[:,-2]

        # What was the direction of the last step taken? 
        LastStep = LastPos - PenUltPos

        # How many different conformations do we have?
        Npos = np.shape(PosArray)[0]


        # Straight: same step as last time
        PStraight = LastPos + LastStep*1

        # Turn left: Multiply by 1j
        PLeft = LastPos + LastStep*1j

        # Turn right: Multiply by -1j 
        PRight = LastPos + LastStep*(-1j)

        # Make an array of new positions
        NewPosArray = np.vstack((np.vstack((PosArray.transpose(), PRight)).transpose(),
                         np.vstack((PosArray.transpose(), PStraight)).transpose(),
                         np.vstack((PosArray.transpose(), PLeft)).transpose()))

        # Check if last position matches any prior ones
        # DispMat is the displacement of the last bead from all other beads
        DispMat = (NewPosArray[:,-1]-NewPosArray[:,0:-1].transpose()).transpose()

        # DistMat is the distance from the last bead to all other beads
        DistMat = np.abs(DispMat)

        # MoveOkay==True if the new bead doesn't conflict with a previous bead
        MoveOkay = (np.min(DistMat,1)>eps)

        PosArray = NewPosArray[MoveOkay,:]


    return PosArray


In [2]:
def plot_conformations(PosArray, Seq, pop=[], ms=6):
    maxVal = np.max((np.real(PosArray), np.imag(PosArray)))
    minVal = np.min((np.real(PosArray), np.imag(PosArray)))
    
    
    # Each conformation gets its own subplot
    Nconf = np.shape(PosArray)[0]
    Nres = np.shape(PosArray)[1]
    if Nres!=len(Seq):
        print('Error! Sequence and position arrays are not the same length!')
        return []
    
    if len(pop)>0 and len(pop)!=Nconf:
        print('Input error: Population vector length does not match conformation list.')
        return []
    
    # Classify residues
    Symb = []
    for n in range(0, Nres):
        if Seq[n]=='A':
            Symb.append('y')
        elif Seq[n]=='K':
            Symb.append('b')
        elif Seq[n]=='E':
            Symb.append('r')
        elif Seq[n]=='P':
            Symb.append('m')
        else:
            print('Error! Unrecognized residue type ' + Seq[n])
            return []
    
    # Generate nearest square figure
    Nrows = np.int(np.ceil(np.sqrt(Nconf)))
    Ncols = Nrows
    
    for p in range(0, Nconf):
        plt.subplot(Nrows, Ncols, p+1)
        plt.plot(np.real(PosArray[p,:]), np.imag(PosArray[p,:]), '-o', markersize=ms, fillstyle='none')
        for n in range(0, Nres):
            plt.plot(np.real(PosArray[p,n]), np.imag(PosArray[p,n]), 'o'+Symb[n], markersize=ms)
        plt.plot(np.real(PosArray[p,0]), np.imag(PosArray[p,0]), 'kx', markersize=ms*1.5, fillstyle='none')
        plt.xlim([minVal-0.25, maxVal+0.25])
        plt.ylim([minVal-0.25, maxVal+0.25])
        ax = plt.gca()
        if len(pop)>0:
            plt.text(minVal, minVal, '{:.1f}'.format(pop[p]) + '%', fontsize=16)
        #plt.axis('off')
        plt.xticks([])
        plt.yticks([])
        
    plt.subplots_adjust(left=0, 
                    bottom=0,  
                    right=1,  
                    top=1,  
                    wspace=0,  
                    hspace=0)

In [3]:
def calculate_energy(PosArray, Seq):
    
    Nconf = np.shape(PosArray)[0]
    Nres = np.shape(PosArray)[1]
    if Nres!=len(Seq):
        print('Error! Sequence and position arrays are not the same length!')
        return []
    
    Energy = np.zeros((Nconf,))
    
    Shift0 = 2*R*300.0
    Ashift = -Shift0
    EKshift = -Shift0
    EEshift = Shift0
    KKshift = Shift0
    Pshift = Shift0
    
    # Classify residues
    AList = []
    KList = []
    EList = []
    PList = []
    for n in range(0, Nres):
        if Seq[n]=='A':
            AList.append(n)
        elif Seq[n]=='K':
            KList.append(n)
        elif Seq[n]=='E':
            EList.append(n)
        elif Seq[n]=='P':
            PList.append(n)
        else:
            print('Error! Unrecognized residue type ' + Seq[n])
            return []
            
            
    # Add Ala-Ala contact energy
    for n1 in range(0, len(AList)):
        for n2 in range(n1+1, len(AList)):
            r1 = AList[n1]
            r2 = AList[n2]
            Dist = np.abs(PosArray[:,r1] - PosArray[:,r2])
            Energy += Ashift*(np.abs(Dist-1)<eps)
                
    # Add Lys-Glu contact energy
    for n1 in range(0, len(EList)):
        for n2 in range(0, len(KList)):
            r1 = EList[n1]
            r2 = KList[n2]
            # If not the same residue and not sequential residues
            if (r1!=r2) and np.abs(r1-r2)>1:
                Dist = np.abs(PosArray[:,r1] - PosArray[:,r2])
                Energy += EKshift*(np.abs(Dist-1)<eps)
                
    # Add Lys-Lys contact energy
    for n1 in range(0, len(KList)):
        for n2 in range(n1, len(KList)):
            r1 = KList[n1]
            r2 = KList[n2]
            # If not the same residue and not sequential residues
            if (r1!=r2) and np.abs(r1-r2)>1:
                Dist = np.abs(PosArray[:,r1] - PosArray[:,r2])
                Energy += KKshift*(np.abs(Dist-1)<eps)
                
    # Add Glu-Glu contact energy
    for n1 in range(0, len(EList)):
        for n2 in range(n1, len(EList)):
            r1 = EList[n1]
            r2 = EList[n2]
            # If not the same residue and not sequential residues
            if (r1!=r2) and np.abs(r1-r2)>1:
                Dist = np.abs(PosArray[:,r1] - PosArray[:,r2])
                Energy += EEshift*(np.abs(Dist-1)<eps)
                
    # Add Pro turn energy
    for n in range(0, len(PList)):
        r = PList[n]
        
        # Terminal Pro's don't affect energy
        if r>=1 and r<Nres-1:
            Move = PosArray[:,r+1] - PosArray[:,r]
            PriorMove = PosArray[:,r] - PosArray[:,r-1]
            
            # Add positive energy for left turns
            Energy += Pshift*(np.abs(Move/PriorMove-1j)<eps)
            
            # Add negative energy for right turns
            Energy -= Pshift*(np.abs(Move/PriorMove+1j)<eps)
    
    return Energy

def calculate_length(PosArray):
    Nconf = np.shape(PosArray)[0]
    Nres = np.shape(PosArray)[1]
    if Nres<2:
        return np.zeros((Nconf,))
    
    else:
        return np.abs(PosArray[:,-1] - PosArray[:,0])
        
def calculate_populations(Energy, T):
    Eo = np.min(Energy)
    Q = np.sum(np.exp(-(Energy-Eo)/(R*T)))
    Pops = np.exp(-(Energy-Eo)/(R*T))/Q
    return Pops, Q

In [1]:
def run_simulation(Seq):
    
    Confs = generate_positions(len(Seq))
    Energy = calculate_energy(Confs, Seq)
    Lengths = calculate_length(Confs)
    
    Taxis = np.arange(10, 350, 10)
    Laxis = []
    Qaxis = []
    for T in Taxis:
        Pops, Q = calculate_populations(Energy, T)
        Lbar = np.sum(Lengths*Pops)
        Laxis.append(Lbar)
        Qaxis.append(Q)
        ndcs = np.argsort(-Pops)
    
    Pops, Q = calculate_populations(Energy, 300)
    ndcs = np.argsort(-Pops)
    
    fig = plt.figure(figsize=(5,5))
    plot_conformations(Confs[ndcs[0:9],:], Seq, pop=Pops[ndcs[0:9]]*100)
    
    ax1 = fig.add_axes([1.2, 0.1, 0.5, 0.4])
    plt.plot([298, 298], [np.min(Qaxis), np.max(Qaxis)], '--', color=[0.5,0.5,0.5], linewidth=1)
    plt.plot(Taxis, Qaxis)
    plt.xlabel('Temperature (K)', fontsize=16)
    plt.xticks(fontsize=16)
    plt.ylabel('$Z\'(T)$', fontsize=16)
    
    ax2 = fig.add_axes([1.2, 0.5, 0.5, 0.4])
    plt.plot([298, 298], [np.min(Laxis), np.max(Laxis)], '--', color=[0.5,0.5,0.5], linewidth=1)
    plt.plot(Taxis, Laxis)
    plt.ylabel('$\\left \\langle L \\right \\rangle$', fontsize=16)
    ax2.xaxis.tick_top()
    ax2.xaxis.set_label_position('top') 
    plt.xlabel('Temperature (K)', fontsize=16)
    plt.xticks(fontsize=16)
    
    #plt.ylim(1, 0.5*len(Seq))
    plt.show()

In [5]:
#run_simulation('KEPAKE')
#run_simulation('KAPPAPPAPPAE')

In [6]:
SeqHeader = "<p style=\"font-size:16pt\">"
SeqTail = "</p>"

SeqLabel_html = widgets.HTML(
    value = SeqHeader + "<b>Sequence:</b>" + SeqTail,
    placeholder='',
)


Seq_html = widgets.HTML(
    value = SeqHeader + '' + SeqTail,
    placeholder='Peptide Sequence',
)

def extract_sequence():
    n1 = Seq_html.value.find(SeqHeader) + len(SeqHeader)
    n2 = Seq_html.value.find(SeqTail)
    return Seq_html.value[n1:n2]
    

MaxLength = 15
def Add_onclick(b):
    if len(extract_sequence())<MaxLength:
        Seq_html.value = SeqHeader + extract_sequence() + b.description + SeqTail
    
def Undo_onclick(b):
    Seq_html.value = SeqHeader + extract_sequence()[:-1] + SeqTail
    
def Clear_onclick(b):
    Seq_html.value = SeqHeader + SeqTail
    
def run_all(ev):
    global go
    go = True
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    
AddK_button = widgets.Button(
    description='K', 
    layout=widgets.Layout(width='0.8cm')
)
AddK_button.on_click(Add_onclick)

AddA_button = widgets.Button(
    description='A', 
    layout=widgets.Layout(width='0.8cm')
)
AddA_button.on_click(Add_onclick)

AddE_button = widgets.Button(
    description='E', 
    layout=widgets.Layout(width='0.8cm')
)
AddE_button.on_click(Add_onclick)

AddP_button = widgets.Button(
    description='P', 
    layout=widgets.Layout(width='0.8cm')
)
AddP_button.on_click(Add_onclick)

Undo_button = widgets.Button(
    description='Undo'
)
Undo_button.on_click(Undo_onclick)

Clear_button = widgets.Button(
    description='Clear'
)
Clear_button.on_click(Clear_onclick)

Run_button = widgets.Button(
    description='Go!'
)
Run_button.on_click(run_all)

AddBox = widgets.HBox([AddA_button, AddE_button, AddK_button, AddP_button, Undo_button, Clear_button])
SeqBox = widgets.HBox([SeqLabel_html, Seq_html])

display(AddBox)
display(SeqBox)
display(Run_button)

<IPython.core.display.Javascript object>

NameError: name 'widgets' is not defined

In [ ]:
if len(extract_sequence())>0:
    display_markdown('## Most Stable Conformations at 298 K ##', raw=True)
    run_simulation(extract_sequence())
    display_markdown('<span style="font-size:14pt">Legend: </span> \
                     <span style="color:#CEC353; font-size:20pt">&#8226;</span><span style="font-size:14pt"> A   </span> \
                     <span style="color:red; font-size:20pt">&#8226;</span><span style="font-size:14pt"> E</span>\
                     <span style="color:blue; font-size:20pt">&#8226;</span><span style="font-size:14pt"> K  </span>\
                     <span style="color:#CE53C9; font-size:20pt">&#8226;</span><span style="font-size:14pt"> P  </span>', raw=True)